# SBIRT GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\GPRA\SBIRT_GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM',
                                                                                                'FORM_UPDATE_DTTM','DOB'])


#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - SBIRT'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

FileNotFoundError: [Errno 2] No such file or directory: '\\\\cdchomedir01\\home1\\tyogarro\\GPRA\\SBIRT_GPRA_Coding_File.csv'

In [ ]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)

In [ ]:
#Inserting BatchID columns
df['BatchID'] = ''

In [ ]:
#function for days columns that exceed 30 days
def exceed_thirty_days(x):
    if x > 30:
        return 30
    else:
        return x

## Creating GPRA 1

In [ ]:
#Creating function to create column ClientDescripTreat
def Client_Descrip_Treat (x):
    if x == 'Treatment grant client':
        return 1
    else:
        return 0 
    
#Creating column ClientDescripTreat via applying function Client_Descrip_Treat
df['ClientDescripTreat'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Treat (x))

In [ ]:
#Creating function to create column ClientDescripRecov
def Client_Descrip_Recov (x):
    if x == 'Client in recovery grant':
        return 1
    else:
        return 0
    
#Creating column ClientDescripRecov via applying function Client_Descrip_Recov
df['ClientDescripRecov'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Recov (x))

In [ ]:
#Creating column GrantNo column
df['GrantNo'] = 'TI084061'

In [ ]:
#Creating function for interview type
def interview_type_code(row):
    if row['INTERVIEW_TYPE'] == 'Intake':
        return 1
    elif row['INTERVIEW_TYPE'] == '6-month follow-up':
        return 2
    elif row['INTERVIEW_TYPE'] == '3-month follow-up':
        return 4
    elif row['INTERVIEW_TYPE'] == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df.apply(interview_type_code, axis = 1)

In [ ]:
#Creating function for conducted interview
def conducted_interview(x):
    if x == '6-month follow-up' or x == '3-month follow-up' or x == 'Discharge' or x == 'Intake':
        return 1
    else:
        return 0
    
# Creating a new column via applying the conducted_interview function to the values in the interview type column
df['ConductedInterview'] = df['INTERVIEW_TYPE'].apply(lambda x: conducted_interview(x))

In [ ]:
#Renaming column to InterviewDate and changing data type to output in the format MM/DD/YYYY
df['InterviewDate'] = df['FORM_DTTM'].dt.strftime('%m/%d/%Y')

def conducted_interview_code(row):
    if row['ConductedInterview'] ==1:
        return row['InterviewDate']
    else:
        pass
    
df['InterviewDate'] = df.apply(conducted_interview_code,axis=1)

In [ ]:
# Rolling Count to create IntakeSeqNum column
df['IntakeSeqNum'] = df.groupby('MRN')['ADMIT_DT'].cumcount() + 1 # <- rolling count +1 to start count at 1

In [ ]:
#Creating InactFlag Column
df['MRN_CNT'] = df.groupby(['PATIENT'])['MRN'].transform('count') # <- will return count of MRN the length of df
df['MAX_ADMIT_DT'] = df.groupby(['PATIENT'])['ADMIT_DT'].transform('max') # <- will return MAX ADMIT_DT the length of df

# Function across multiple columns
def inactflag (row):
    if row['MRN_CNT'] == 1:
        return 0
    elif row['MRN_CNT'] > 1 and row['ADMIT_DT'] == row['MAX_ADMIT_DT']:
        return 0
    else:
        return 1
df['InactFlag'] = df.apply(inactflag, axis =1 ) # <- axis 1 will apply function across columns 

In [ ]:
df['InactFlag'].unique()

In [ ]:
# Assigning 0 for active grants
df['GrantInactFlag'] = 0

In [ ]:
#Extracting year from ADMIT_DT to create column FFY
df['FFY'] = df['FORM_DTTM'].dt.year

#Extracting year from ADMIT_DT to create column Month
df['Month'] =  df['FORM_DTTM'].dt.month

#Extracting year from ADMIT_DT to create column Quarter

#Creating a function for Quarter
def year_qt (row):
    if row['Month'] in [10,11,12]: # <- in kwarg that references a list 
        return 1
    elif row['Month'] in [1,2,3]:
        return 2
    elif row['Month'] in [4,5,6]:
        return 3
    elif row['Month'] in [7,8,9]:
        return 4    

df['Quarter'] =  df.apply(year_qt, axis = 1)

# Convering DOB to datetime format
df['DOB'] = pd.to_datetime(df['DOB'])


#BirthMonth
df['BirthMonth'] = df['DOB'].dt.month

#BirthYear
df['BirthYear'] = df['DOB'].dt.year

#Age
df['Age'] = df['FFY'] - df['BirthYear']


#AgeGroup
def age_group(row):# <- creating conditions for intake
    if row['Age'] >= 18 and row['Age'] < 25:
        return 1
    elif row['Age'] >= 25 and row['Age'] < 35:
        return 3
    elif row['Age'] >= 35 and row['Age'] < 45:
        return 4
    elif  row['Age'] >= 45 and row['Age'] < 55:
        return 5
    elif row['Age'] >=55 and row['Age'] < 65:
        return 6
    elif row['Age'] >= 65:
        return 7
    else:
        return -9
    
df['AgeGroup'] = df.apply(age_group,axis=1)

In [ ]:
#Creating function for gender code 
def gender_code(row):
    if row['GENDER'] == 'Male':
        return 1
    elif row['GENDER'] == 'Female':
        return 2
    else:
        return 7
    
df['Gender'] = df.apply(gender_code,axis=1)

#Renaming column to create column GenderSpec
df.rename(columns = {'WHAT_DO_YOU_CONSIDER_YOURSELF_TO_BE_OTHER':'GenderSpec'}, inplace = True)

In [ ]:
#Creating a function for HispanicLatino Column
def Hispanic_Latino (row):
    if row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'Yes':
        return 1
    elif row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'No (Skip to Question 3)':
        return 0
    elif  row['HISPANIC_LATIN_SPANISH_ORIGIN'] == 'REFUSED (Skip to Question 3)':
        return -7
    else:
        return -9

    
df['HispanicLatino'] = df.apply(Hispanic_Latino,axis=1)

In [ ]:
#Inputing 'NaN' for missing values in the WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF column
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].fillna('NaN')

#Creating substrings
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].str.split(',').str[0]

#Creating a function for column EthnicCentranAmerican
def Central_American(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Central American':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0


df['EthnicCentralAmerican'] = df.apply(Central_American,axis=1)

In [ ]:
#Creating a function for column EthnicCuban
def Cuban(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Cuban':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    

df['EthnicCuban'] = df.apply(Cuban,axis=1)

In [ ]:
#Creating a function for column EthnicDominican
def DR(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Dominican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0


df['EthnicDominican'] = df.apply(DR,axis=1)

In [ ]:
 #Creating a function for column EthnicMexican
def Mexican(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Mexican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0


df['EthnicMexican'] = df.apply(Mexican,axis=1)

In [ ]:
#Creating a function for column EthnicPuertoRican
def PR(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Puerto Rican':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    
df['EthnicPuertoRican'] =  df.apply(PR,axis=1)

In [ ]:
#Creating a function for column EthnicSouthAmerican
def South_America(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'South American':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    
df['EthnicSouthAmerican'] = df.apply(South_America,axis=1)    

In [ ]:
#Creating a function for column EthnicOther
def Ethnic_Other(row):
    if row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'Other (SPECIFY)':
        return 1
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] ==  'REFUSED':
        return -7
    elif row['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] == 'NaN':
        return -1
    else:
        return 0
    

df['EthnicOther'] = df.apply(Ethnic_Other,axis=1)

#Renaming column
df.rename(columns = {'ETHNIC_GROUP_OTHER':'EthnicOtherSpec'}, inplace = True)

In [ ]:
#Fill null values with 'NaN' in the Race column
df['RACE'] = df['RACE'].fillna('NaN')

#Creating substrings 
df['RACE'] = df['RACE'].str.split(',').str[0]

#Creating function for column RaceBlack
def Race_Black(row):
    if row['RACE'] == 'Black or African American':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    elif row['RACE'] == 'Other (SPECIFY), REFUSED':
        return -7
    else:
        return 0
    
df['RaceBlack'] = df.apply(Race_Black,axis=1)

In [ ]:
#Creating function for column RaceWhite
def Race_White(row):
    if row['RACE'] == 'White':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

df['RaceWhite'] = df.apply(Race_White,axis=1)

In [ ]:
#Creating function for column RaceAmericanIndian
def Race_Am_Indian(row):
    if row['RACE'] == 'American Indian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

    
df['RaceAmericanIndian'] = df.apply(Race_Am_Indian,axis=1) 

In [ ]:
#Creating function for column RaceAlaskaNative
def Race_Alaska_Native(row):
    if row['RACE'] == 'Alaska Native':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
    
df['RaceAlaskaNative'] = df.apply(Race_Alaska_Native,axis=1)

In [ ]:
df['RaceAlaskaNative']

In [ ]:
#Creating function for column RaceAsianIndian
def Race_Asian_Indian(row):
    if row['RACE']== 'Asian Indian':
        return 1
    elif row['RACE']== 'NaN':
        return -9
    elif row['RACE']== 'REFUSED':
        return -7
    else:
        return 0
    
    
df['RaceAsianIndian'] = df.apply(Race_Asian_Indian,axis=1) 

In [ ]:
#Creating function for column RaceChinese
def Race_Chinese(row):
    if row['RACE'] == 'Chinese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

    
df['RaceChinese'] = df.apply(Race_Chinese,axis=1)

In [ ]:
#Creating function for column RaceFilipino
def Race_Filipino(row):
    if row['RACE'] == 'Filipino':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0


df['RaceFilipino'] = df.apply(Race_Filipino,axis=1)

In [ ]:
#Creating function for column RaceJapanese
def Race_Japanese(row):
    if row['RACE'] == 'Japanese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
    
df['RaceJapanese'] = df.apply(Race_Japanese,axis=1)

In [ ]:
#Creating function for column RaceKorean
def Race_Korean(row):
    if row['RACE'] == 'Korean':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    
    
df['RaceKorean'] = df.apply(Race_Korean,axis=1)

In [ ]:
#Creating function for column RaceVietnamese
def Race_Vietnamese(row):
    if row['RACE'] == 'Vietnamese':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

    
df['RaceVietnamese'] = df.apply(Race_Vietnamese,axis=1)

In [ ]:
#Creating function for column RaceOtherAsian
def Race_Other_Asian(row):
    if row['RACE'] == 'Other Asian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

    
df['RaceOtherAsian'] = df.apply(Race_Other_Asian,axis=1) 

In [ ]:
#Creating function for column RaceNativeHawaiian
def Race_Hawaiian(row):
    if row['RACE'] == 'Hawaiian':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

df['RaceNativeHawaiian'] = df.apply(Race_Hawaiian,axis=1)

In [ ]:
#Creating function for column RaceGuamanianChamorro
def Race_Guamanian_Chamorro(row):
    if row['RACE'] == 'Guamanian Chamorro':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0

    
df['RaceGuamanianChamorro'] = df.apply(Race_Guamanian_Chamorro,axis=1)

In [ ]:
#Creating function for column RaceSamoan
def Race_Samoan(row):
    if row['RACE'] == 'Samoan':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    

df['RaceSamoan'] = df.apply(Race_Samoan,axis=1)

In [ ]:
#Creating function for column RaceOtherPacificIslander
def Race_Pacific_Islander(row):
    if row['RACE'] == 'Pacific Islander':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0


df['RaceOtherPacificIslander'] = df.apply(Race_Pacific_Islander,axis=1)

In [ ]:
#Creating function for column RaceOther
def Race_Other(row):
    if row['RACE'] == 'Other (SPECIFY)':
        return 1
    elif row['RACE'] == 'NaN':
        return -9
    elif row['RACE'] == 'REFUSED':
        return -7
    else:
        return 0
    

df['RaceOther'] = df.apply(Race_Other,axis=1)

#Renaming column 
df.rename(columns = {'RACE_OTHER':'RaceSpec'}, inplace = True)

#Renaming column to create column RaceSpec
df['RaceSpec'] = df['RaceSpec']


In [ ]:
def lang_other_span(row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return 'Yes'
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'Yes' and row['WHAT_OTHER_LANGUAGE'] == 'Spanish':
        return 'No'
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'Yes' and row['WHAT_OTHER_LANGUAGE'] == 'Other (SPECIFY)':
        return 'Yes'
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'REFUSED (Skip to Question 5)':
        return row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH']
    
df['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] = df.apply(lang_other_span, axis = 1)

In [ ]:
# Filling missing in values in SPEAK_LANGUAGE_OTHER_THAN_ENGLISH column with 'NaN'
df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] = df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'].fillna('NaN')

#Creating function for LangNotEnglishAtHome
def Lang_Eng_Not_Spoken(row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'NaN':
        return -9


df['LangNotEnglishAtHome'] = df.apply(Lang_Eng_Not_Spoken,axis=1)

#Creating function to create column LangNotEnglishSpoken
def Lang_Not_English_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Spanish':
        return 1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Other (SPECIFY)':
        return 2
    else:
        return -1

    
df['LangNotEnglishSpoken'] = df.apply(Lang_Not_English_Spoken,axis=1)

#Renaming column to create column LangNotEnglishSpokenSpec
df.rename(columns = {'LANGUAGE_OTHER':'LangNotEnglishSpokenSpec'}, inplace = True)

#Creating LangNotSpanishSpokenSpec column
df['LangNotSpanishSpokenSpec'] = ''

In [ ]:
#Creating function to create column LangNotSpanishAtHome
def Lang_Not_Spanish_Spoken_Home (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes' or row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['LangNotEnglishSpoken'] == 2:
        return -1
    elif row['LangNotEnglishAtHome'] == 0:
        return -1
    else:
        return -1
    
    
df['LangNotSpanishAtHome'] = df.apply(Lang_Not_Spanish_Spoken_Home,axis=1)  



#Creating function to create column LangNotSpanishSpoken
def Lang_Not_Spanish_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 2
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['LangNotEnglishAtHome'] == 0:
        return -1
    elif row['LangNotEnglishSpoken'] == 2:
        return -1
    
    else:
        return -1

    
df['LangNotSpanishSpoken'] = df.apply(Lang_Not_Spanish_Spoken,axis=1) 


In [ ]:
#filling missing values with 'NaN'
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].fillna('NaN')

#Creating substrings
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].str.split(',').str[0]

#Creating function for column SexIdentHeterosexual
def Straight (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Straight or Heterosexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
    
df['SexIdentHeterosexual'] = df.apply(Straight,axis=1)

In [ ]:
#Creating function for column SexIdentHomoSexual
def Homosexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Homosexual (Gay or Lesbian)':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0

df['SexIdentHomosexual'] = df.apply(Homosexual,axis=1)

In [ ]:
#Creating function for column SexIdentBiSexual
def Bisexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Bisexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0

df['SexIdentBisexual'] = df.apply(Bisexual,axis=1)

In [ ]:
#Creating function for column SexIdentQueerPanaQuest
def Queer_Pan_Quest (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Queer, Pansexual, And/Or Questioning':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
    
df['SexIdentQueerPanaQuest'] = df.apply(Queer_Pan_Quest,axis=1)

In [ ]:
#Creating function for column SexIdentAsexual
def Asexual (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Asexual':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0

    
df['SexIdentAsexual'] = df.apply(Asexual,axis=1)

In [ ]:
#Creating function for column SexIdentOther
def Other (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Other':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0

    
df['SexIdentOther'] = df.apply(Other,axis=1)

In [ ]:
#Creating function for column SexIdentSpec
def Other_Spec (row):
    if row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'Other (SPECIFY)':
        return 1
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'REFUSED':
        return -7
    elif row['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] == 'NaN':
        return -9
    else:
        return 0
    
def Other_Spec_code(row):
    if row['InterviewType']==1 and row['SexIdentOther'] == 1:
        return Other_Spec(row)
    else:
        pass
    
df['SexIdentSpec'] = df.apply(Other_Spec_code,axis=1)  

In [ ]:
#Filling in null values with'NaN'
df['RELATIONSHIP_STATUS'] = df['RELATIONSHIP_STATUS'].fillna('NaN')

#Creating function for RelationshipStatus column
def Relationship_Status (row):
    if row['RELATIONSHIP_STATUS'] == 'Married':
        return 1
    elif row['RELATIONSHIP_STATUS'] == 'Single':
        return 2
    elif row['RELATIONSHIP_STATUS'] == 'Divorced':
        return 3
    elif row['RELATIONSHIP_STATUS'] == 'Separated':
        return 4
    elif row['RELATIONSHIP_STATUS'] == 'Widowed':
        return 5
    elif row['RELATIONSHIP_STATUS'] == 'In a relationship':
        return 6
    elif row['RELATIONSHIP_STATUS'] == 'In multiple relationships':
        return 7
    elif row['RELATIONSHIP_STATUS'] == 'REFUSED':
        return -7
    elif row['RELATIONSHIP_STATUS'] == 'NaN':
        return -9

df['RelationshipStatus'] = df.apply(Relationship_Status,axis=1)


In [ ]:
#Filling missing values with 'NaN'
df['CURRENTLY_PREGNANT'] = df['CURRENTLY_PREGNANT'].fillna('NaN')

#Creating function for Pregnant column
def Pregnant (row):
    if row['CURRENTLY_PREGNANT'] == 'Yes':
        return 1
    elif row['CURRENTLY_PREGNANT'] == 'No':
        return 0
    elif row['CURRENTLY_PREGNANT'] == 'Do not know':
        return -8
    elif row['CURRENTLY_PREGNANT'] == 'REFUSED':
        return -7
    elif row['CURRENTLY_PREGNANT'] == 'NaN':
        return -9

df['Pregnant'] = df.apply(Pregnant,axis=1)

In [ ]:
#Replacing null values with 'NaN'
df['HAVE_CHILDREN'] = df['HAVE_CHILDREN'].fillna(-9)

# Creating function to create new column Children
def Child (row):
    if row['HAVE_CHILDREN'] == 'Yes':
        return 1
    elif row['HAVE_CHILDREN'] == 'No (Skip to Question 9)':
        return 0
    elif row['HAVE_CHILDREN'] == 'REFUSED (Skip to Question 9)':
        return -7
    elif row['HAVE_CHILDREN'] == -9:
        return -9

df['Children'] = df.apply(Child,axis=1)

In [ ]:
#Renaming columns 
df.rename(columns = {'CHILDREN_UNDER_18':'ChildrenUnder18Nr'}, inplace = True)


def ChildrenUnder18Nr(row):
    if row['Children'] == 1:
        return row['ChildrenUnder18Nr']
    else:
        pass
    
df['ChildrenUnder18Nr'] = df.apply(ChildrenUnder18Nr,axis=1)

In [ ]:
#Creating function to create column ChildrenCustody
def custody(row):
    if row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'Yes':
        return 1
    elif  row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'No (Skip to Question 9)':
        return 0

def Children_Custody_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenUnder18Nr'] > 0:
        return custody(row)
    
    
    
df['ChildrenCustody'] = df.apply(Children_Custody_code,axis=1)

In [ ]:
#Renaming column 
df.rename(columns = {'CHILDREN_REMOVED_FROM_CLIENTS_CARE':'ChildrenCustodyNr'}, inplace = True)

def custody_nr (row):
    if row['ChildrenCustody'] == 1:
        return row['ChildrenCustodyNr'] 

def ChildrenCustodyNr(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    else:
        return custody_nr(row)

df['ChildrenCustodyNr'] = df.apply(ChildrenCustodyNr,axis=1)

In [ ]:
#Creating function to create column ChildrenReunited
def child_reunited(row):
    if row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE'] == 'Yes':
        return 1
    elif row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE']== 'No':
        return 0

def Children_Reunited_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenCustody'] != 1:
        pass
    else:
        return child_reunited(row)

    
df['ChildrenReunited'] = df.apply(Children_Reunited_code,axis=1)

In [ ]:
#Renaming column 
df.rename(columns = {'CHILDREN_REUNITED_WITH_CLIENT':'ChildrenReunitedNr'},inplace = True)

def ChildrenReunitedNr(row):
    if row['ChildrenReunited'] == 1:
        return row['ChildrenReunitedNr']

df['ChildrenReunitedNr'] = df.apply(ChildrenReunitedNr,axis=1)

In [ ]:
#Replacing null values 'NaN'
df['EVER_SERVED_IN_MILITARY'] = df['EVER_SERVED_IN_MILITARY'].fillna('NaN')

#Creating function to create new column MilitaryServed
def Military_Served (row):
    if row['EVER_SERVED_IN_MILITARY'] == 'No':
        return 0
    elif row['EVER_SERVED_IN_MILITARY'] == 'Yes, In the Armed Forces':
        return 1
    elif row['EVER_SERVED_IN_MILITARY'] == 'Yes, In the National Guard':
        return 3
    elif row['EVER_SERVED_IN_MILITARY'] == 'REFUSED':
        return -7
    elif row['EVER_SERVED_IN_MILITARY'] == 'NaN':
        return -9

    
df['MilitaryServed'] = df.apply(Military_Served,axis=1)

In [ ]:
#Replacing null values 'NaN'
df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] = df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'].fillna('NaN')

#Creating function to create new column ServicesTravelTime
def Travel_Time (row):
    if row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Half an hour or less':
        return 1
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between half an hour and one hour':
        return 2
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between one hour and one and a half hours':
        return 3
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Between one and a half hours and two hours':
        return 4
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'Two hours or more':
        return 5
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'REFUSED':
        return -7
    elif row['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] == 'NaN':
        return -9


    
df['ServicesTravelTime'] = df.apply(Travel_Time,axis=1)

In [ ]:
# creating a function for the column Sex
def gender(row):
    if row['WHAT_DO_YOU_CONSIDER_YOURSELF_TO_BE'] == 'Male':
        return 1
    elif row['WHAT_DO_YOU_CONSIDER_YOURSELF_TO_BE'] == 'Female':
        return 2
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return gender(row)
    else:
        pass
    
df['Sex'] = df.apply(interview_type, axis = 1)

In [ ]:
# filtering columnms based on Interview Type

# list object with columns to be filtered
filter_columns = ['BirthMonth','BirthYear','Age','AgeGroup',
                  'Gender','GenderSpec','HispanicLatino','EthnicCentralAmerican',
                  'EthnicCuban','EthnicDominican','EthnicMexican','EthnicPuertoRican',
                  'EthnicSouthAmerican','EthnicOther','EthnicOtherSpec','RaceBlack','RaceWhite',
                  'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian','RaceChinese','RaceFilipino',
                  'RaceJapanese','RaceKorean','RaceVietnamese','RaceOtherAsian','RaceNativeHawaiian',
                  'RaceGuamanianChamorro','RaceSamoan','RaceOtherPacificIslander','RaceOther','RaceSpec',
                  'LangNotEnglishAtHome','LangNotEnglishSpoken','LangNotEnglishSpokenSpec','LangNotSpanishAtHome',
                  'LangNotSpanishSpoken','LangNotSpanishSpokenSpec','SexIdentHeterosexual','SexIdentHomosexual',
                  'SexIdentBisexual','SexIdentQueerPanaQuest','SexIdentAsexual','SexIdentOther','SexIdentSpec',
                  'RelationshipStatus','Pregnant','Children','ChildrenUnder18Nr','ChildrenCustody','ChildrenCustodyNr',
                  'ChildrenReunited','ChildrenReunitedNr','MilitaryServed','ServicesTravelTime']

def interview_type(row):
    if row['InterviewType'] == 1:
        return row[filter_columns]
    else:
        pass
    
df[filter_columns] = df.apply(interview_type, axis = 1)

In [ ]:
df.shape

In [ ]:
df[filter_columns].shape

In [ ]:
#Renaming column
df.rename(columns = {'ALCOHOL_NUMBER_OF_DAYS':'AlcoholDays'}, inplace = True)

#Replacing null values with 0 to create column AlcoholDays
df['AlcoholDays'] = df['AlcoholDays'].fillna(0)

#Replacing values greater than 30 with 30
df['AlcoholDays'] = df['AlcoholDays'].apply(lambda x:exceed_thirty_days(x))

In [ ]:
#Creating function for AlcoholRoute
def Alcohol_Route (row):
    if row['ALCOHOL_ROUTE'] == '1. Oral':
        return 1
    elif row['AlcoholDays'] ==0:
        return -1
    else:
        return -1
    
#Creating column AlcoholRoute via applying the function Alcohol_Route
df['AlcoholRoute'] = df.apply(Alcohol_Route, axis = 1 )

In [ ]:
#Creating column AlcoholOther1Days 
df['AlcoholOther1Days'] = 0

#Creating column AlcoholOther1Route 
df['AlcoholOther1Route'] = -1

df['AlcoholOther1Spec'] = " "

#Replacing missing values to create respective columns
df['AlcoholOther2Days'] = 0
df['AlcoholOther2Route'] = -1
df['AlcoholOther2Spec'] = " "
df['AlcoholOther3Days'] = 0
df['AlcoholOther3Route'] = -1
df['AlcoholOther3Spec'] = " "

In [ ]:
#Renaming colulmn
df.rename(columns = {'HEROIN_NUMBER_OF_DAYS':'OpioidsHeroinDays'}, inplace = True)

#Replacing null values with 0 to create column OpioidsHeroinDays
df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].fillna(0)

df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].apply(lambda x:exceed_thirty_days(x))

In [ ]:
df['HEROIN_ROUTE']

In [ ]:
#Creating function for column OpioidsHeroinRoute
def Heroin_Route (row):
    if row['HEROIN_ROUTE'] == '1. Oral':
        return 1
    elif row['HEROIN_ROUTE'] == '2. Intranasal':
        return 2
    elif row['HEROIN_ROUTE'] == '3. Vaping':
        return 6
    elif row['HEROIN_ROUTE'] == '5. Non-IV Injection':
        return 4
    elif row['HEROIN_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['HEROIN_ROUTE'] == '0. Other':
        return 0
    elif row['HEROIN_ROUTE'] == '4. Smoking':
        return 3
    elif row['OpioidsHeroinDays'] == 0:
        return -1
    else:
        return -1
    
#Creating column OpioidsHeroinRoute via applying function Heroin_Route
df['OpioidsHeroinRoute'] = df.apply(Heroin_Route, axis = 1)

In [ ]:
df['HEROIN_ROUTE'].unique()

In [ ]:
#Renaming columns
df.rename(columns = {'MORPHINE_NUMBER_OF_DAYS':'OpioidsMorphineDays'}, inplace = True)

# Amending 'MORPHINE_NUMBER_OF_DAYS' column via replacing null values with 0 to create OpioidsMorphineDays column
df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].fillna(0)

df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].apply(lambda x: exceed_thirty_days(x))

In [ ]:
#Creating function to create new column OpioidsMorphineRoute
def Morphine_Route (row):
    if row['MORPHINE_ROUTE'] == '1. Oral':
        return 1
    elif row['MORPHINE_ROUTE'] == '6. Intravenous (IV) Injection':
        return 2
    elif row['MORPHINE_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsMorphineDays'] == 0:
        return -1
    
#Creating new column OpioidsMorphineRoute via applying the function Morphine_Route
df['OpioidsMorphineRoute'] = df.apply(Morphine_Route, axis = 1)

In [ ]:
#Renaming columns 
df.rename(columns = {'FENTANYL_NUMBER_OF_DAYS':'OpioidsFentanylDays'},inplace = True)

# Replacing null values with 0 to create OpioidsFentanylDays column 
df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].fillna(0)

df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].apply(lambda x:exceed_thirty_days(x))

In [ ]:
#Creating function to create new column OpioidsFentanylRoute
def Fentanyl_Route (row):
    if row['FENTANYL_ROUTE'] == '1. Oral':
        return 1
    elif row['FENTANYL_ROUTE'] == '2. Intranasal':
        return 2
    elif row['FENTANYL_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['FENTANYL_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsFentanylDays']==0:
        return -1
    else:
        return -1
    
#Creating new column OpioidsFentanylRoute via applying the function Fentanyl_Route
df['OpioidsFentanylRoute'] = df.apply(Fentanyl_Route, axis = 1)

In [ ]:
#Renaming column
df.rename(columns = {'DILAUDID_NUMBER_OF_DAYS':'OpioidsDilaudidDays'}, inplace = True)

#Replacing null values with 0 to create values for column OpioidsDilaudidDays
df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].fillna(0)

df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].apply(lambda x:exceed_thirty_days(x))

In [ ]:
#Creating function for creating column OpioidsDilaudidRoute
def Dilaudid_Route (row):
    if row ['DILAUDID_ROUTE'] == '1. Oral':
        return 1
    elif row['OpioidsDilaudidDays'] == 0:
        return -1
    else:
        return -1

# Creating new column OpioidsDilaudidRoutev via applying function Dilaudid_Route  
df['OpioidsDilaudidRoute'] = df.apply(Dilaudid_Route,axis = 1)

In [ ]:
#Renaming column
df.rename(columns = {'DEMEROL_NUMBER_OF_DAYS':'OpioidsDemerolDays'}, inplace = True)

#Replacing nulll values with 0 to create values for column OpioidsDemerolDays
df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].fillna(0)

df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].apply(lambda x: exceed_thirty_days(x))

## Exporting GPRA SBIRT 1


In [ ]:

#Create variable by indexing columns and creating df
GPRA1_SBIRT = df[['BatchID','PATIENT','PROVIDER_COUNTY','MRN','ClientID','ClientDescripTreat',
                     'ClientDescripRecov','GrantNo',	'InterviewType',	'ConductedInterview',
                     'InterviewDate',	'IntakeSeqNum',	'GrantInactFlag',	'InactFlag',	'FFY',	'Quarter',
                     'Month',	'BirthMonth',	'BirthYear',	'Age',	'AgeGroup',	'Gender',	'GenderSpec',
                     'HispanicLatino',	'EthnicCentralAmerican',	'EthnicCuban',	'EthnicDominican',	'EthnicMexican',
                     'EthnicPuertoRican',	'EthnicSouthAmerican',	'EthnicOther',	'EthnicOtherSpec',	'RaceBlack',	
                     'RaceWhite',	'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian',	'RaceChinese',	
                     'RaceFilipino',	'RaceJapanese',	'RaceKorean',	'RaceVietnamese',	'RaceOtherAsian',	
                     'RaceNativeHawaiian',	'RaceGuamanianChamorro',	'RaceSamoan',	'RaceOtherPacificIslander',	
                     'RaceOther',	'RaceSpec',	'LangNotEnglishAtHome',	'LangNotEnglishSpoken',	'LangNotEnglishSpokenSpec',
                     'LangNotSpanishAtHome',	'LangNotSpanishSpoken',	'LangNotSpanishSpokenSpec',	'SexIdentHeterosexual',
                     'SexIdentHomosexual',	'SexIdentBisexual',	'SexIdentQueerPanaQuest',	'SexIdentAsexual',	
                     'SexIdentOther',	'SexIdentSpec',	'RelationshipStatus',	'Pregnant',	'Children',	
                     'ChildrenUnder18Nr',	'ChildrenCustody',	'ChildrenCustodyNr',	'ChildrenReunited',	
                     'ChildrenReunitedNr',	'MilitaryServed',	'ServicesTravelTime','Sex','AlcoholDays',	'AlcoholRoute',
                     'AlcoholOther1Days',	'AlcoholOther1Route',	'AlcoholOther1Spec',	'AlcoholOther2Days',
                     'AlcoholOther2Route',	'AlcoholOther2Spec',	'AlcoholOther3Days',	'AlcoholOther3Route',
                     'AlcoholOther3Spec',	'OpioidsHeroinDays',	'OpioidsHeroinRoute',	'OpioidsMorphineDays',
                     'OpioidsMorphineRoute',	'OpioidsFentanylDays',	'OpioidsFentanylRoute',	'OpioidsDilaudidDays',
                     'OpioidsDilaudidRoute',	'OpioidsDemerolDays']]

#Exporting GPRA 1
GPRA1_SBIRT.to_csv('GPRA1_SBIRT.csv', index = False, header = True, encoding = 'utf-8')